### 설문조사 자료 전처리
    
    최초 작성일 : 20/03/02
    작성자 : 김은주
    
    작성내용 : 가설검정 용 설문조사 데이터 전처리
    
               가설 검정 항목에 따른 설문조사 진행
               (퍼스널컬러 가설검증 항목.ipynb)
               - 밝아보이는 색 설문
                 http://naver.me/xY6Jjiez(네이버 폼 활용) : 박재중, 양희승, 조예슬
               - 조화로운 색 설문
                 http://naver.me/xs4ePFU7(네이버 폼 활용) : 장윤성, 김도연, 김은주
               
               네이버폼 응답결과 데이터 전처리
               - spread sheet로 저장, 불러오기
               -> 기본항목 데이터 전처리
               -> 측색부위 데이터 전처리 / 2020.03.02 작성완료
               -> 퍼스널컬러 팔레트 데이터 전처리 (색고정, 명도/채도변경)
               -> 퍼스널컬러 팔레트 데이터 전처리 (색변경, 명도/채도고정)
               
               설문 종료 일정 : 2020. 03. 04
               -> spread sheet 업데이트
               -> 전처리 데이터 csv 저장 후 팀원 공유 예쩡

문항|조사내용|조사결과|전처리|컬럼명|값|비고
:---:|:---|:---:|:---:|:---:|:---:|:---
01|화면 밝기조정 안내||→|||컬럼 삭제
02|성별|남성, 여성|→|sex|Male, Female|
03|연령대|10대, 20대, 30대, 40대, 50대이상|→|age_class|10, 20, 30, 40, 50|
04|퍼스널컬러 정의 인식|밝은색, 조화색|→|definition|bright, harmony|
05|퍼스널컬러 필요 이유 인식|밝은색, 조화색|→|need|bright, harmony|
06|퍼스널컬러 사용 목적|옷, 화장, 기타|→|purpose|clothes, cosmetic, etc|
07|설문자 패션 관심도|1, 2, 3, 4, 5|→|interest|1, 2, 3, 4, 5|변경없음
08|측색 안면부위 특정을 위한 항목_1|①, ②, ③, ④, ⑤|→|face region 1|cheek, chin, face, eyes, forehead|
09|측색 안면부위 특정을 위한 항목_2|①, ②, ③, ④, ⑤|→|face region 2|cheek, chin, face, eyes, forehead|
10|측색 안면부위 특정을 위한 항목_3|①, ②, ③, ④, ⑤|→|face region 3|cheek, chin, face, eyes, forehead|
11|측색 안면부위 특정을 위한 항목_4|①, ②, ③, ④, ⑤|→|face region 4|cheek, chin, face, eyes, forehead|
12|피부색과 퍼스널컬러(보색) 비교<br>(색 고정, 명도/채도 변경)|①, ②, ③, ④, ⑤|→|h-f comp|1, 2, 3, 4, 5|모집단분리<br>1. 밝은색 설문<br>2. 조화색 설문
13|피부색과 퍼스널컬러(삼각보색1) 비교<br>(색 고정, 명도/채도 변경)|①, ②, ③, ④, ⑤|→|h-f triangle comp 1|1, 2, 3, 4, 5|h-f : hue fixed
14|피부색과 퍼스널컬러(삼각보색2) 비교<br>(색 고정, 명도/채도 변경)|①, ②, ③, ④, ⑤|→|h-f triangle comp 2|1, 2, 3, 4, 5|
15|피부색과 퍼스널컬러(이중보색1) 비교<br>(색 고정, 명도/채도 변경)|①, ②, ③, ④, ⑤|→|h-f doubleness comp 1|1, 2, 3, 4, 5|
16|피부색과 퍼스널컬러(이중보색2) 비교<br>(색 고정, 명도/채도 변경)|①, ②, ③, ④, ⑤|→|h-f doubleness comp 2|1, 2, 3, 4, 5|
17|피부색과 퍼스널컬러(이중보색3) 비교<br>(색 고정, 명도/채도 변경)|①, ②, ③, ④, ⑤|→|h-f doubleness comp 3|1, 2, 3, 4, 5|
18|피부색과 퍼스널컬러(보색) 비교<br>(색 변경, 명도/채도 고정)|①, ②, ③, ④, ⑤|→|sv-f comp|1, 2, 3, 4, 5|모집단분리<br>1. 밝은색 설문<br>2. 조화색 설문
19|피부색과 퍼스널컬러(삼각보색1) 비교<br>(색 변경, 명도/채도 고정)|①, ②, ③, ④, ⑤|→|sv-f triangle comp 1|1, 2, 3, 4, 5|sv-f : saturation, value fixed
20|피부색과 퍼스널컬러(삼각보색2) 비교<br>(색 변경, 명도/채도 고정)|①, ②, ③, ④, ⑤|→|sv-f triangle comp 2|1, 2, 3, 4, 5|
21|피부색과 퍼스널컬러(이중보색1) 비교<br>(색 변경, 명도/채도 고정)|①, ②, ③, ④, ⑤|→|sv-f doubleness comp 1|1, 2, 3, 4, 5|
22|피부색과 퍼스널컬러(이중보색2) 비교<br>(색 변경, 명도/채도 고정)|①, ②, ③, ④, ⑤|→|sv-f doubleness comp 2|1, 2, 3, 4, 5|
23|피부색과 퍼스널컬러(이중보색3) 비교<br>(색 변경, 명도/채도 고정)|①, ②, ③, ④, ⑤|→|sv-f doubleness comp 3|1, 2, 3, 4, 5|


    수정내용 
        20/03/02

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname='C:\Windows/Fonts/malgun.ttf').get_name()
plt.rc('font', family=font_name)
mpl.rcParams["axes.unicode_minus"] = False

import pymysql, math, time
from sklearn import preprocessing
import urllib

### 기본항목 설문 결과 전처리

In [ ]:
# 기본항목 설문결과 (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
elementaryItem1 = pd.read_excel("data/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D, E, F, G, H")
elementaryItem1.rename(columns={elementaryItem1.columns[0]:"sex",
                          elementaryItem1.columns[1]:"age_class",
                          elementaryItem1.columns[2]:"definition",
                          elementaryItem1.columns[3]:"need",
                          elementaryItem1.columns[4]:"purpose",
                          elementaryItem1.columns[5]:"interest"}, inplace=True)

## 조화색 설문 결과 불러오기
elementaryItem2 = pd.read_excel("data/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D, E, F, G, H")
elementaryItem2.rename(columns={elementaryItem2.columns[0]:"sex",
                          elementaryItem2.columns[1]:"age_class",
                          elementaryItem2.columns[2]:"definition",
                          elementaryItem2.columns[3]:"need",
                          elementaryItem2.columns[4]:"purpose",
                          elementaryItem2.columns[5]:"interest"}, inplace=True)


## 두 결과 합치기
elementaryItem = pd.concat([elementaryItem1, elementaryItem2], join='outer')


## 인덱스 조정
elementaryItem.reset_index(inplace=True)
del elementaryItem['index']


## 값 변경 반복문
sex, age, definition, need, purpose = [], [], [], [], []
for i in range(int(elementaryItem.shape[0])):
    ## sex : Male, Female 리스트 담기
    if elementaryItem["sex"][i] == "남성":
        elementaryItem["sex"][i] = "Male"
    else:
        elementaryItem["sex"][i] = "Female"
        
    ## age_class : 10, 20, 30, 40, 50으로 리스트 담기
    if elementaryItem["age_class"][i] == "10대":
        elementaryItem["age_class"][i] = "10"
    elif elementaryItem["age_class"][i] == "20대":
        elementaryItem["age_class"][i] = "20"
    elif elementaryItem["age_class"][i] == "30대":
        elementaryItem["age_class"][i] = "30"
    elif elementaryItem["age_class"][i] == "40대":
        elementaryItem["age_class"][i] = "40"
    else:
        elementaryItem["age_class"][i] = "50"
    
    ## definition : bright, harmony로 리스트 담기
    if ("생기" in elementaryItem["definition"][i]):
        elementaryItem["definition"][i] = "bright"
    else:
        elementaryItem["definition"][i] = "harmony"
    
    ## need :  bright, harmony로 리스트 담기
    if ("생기" in elementaryItem["need"][i]):
        elementaryItem["need"][i] = "bright"
    else:
        elementaryItem["need"][i] = "harmony"
    
    ## purpose : clothes, cosmetic, etc로 리스트 담기
    if elementaryItem["purpose"][i] == "옷을 구매하는 경우에 활용":
        elementaryItem["purpose"][i] = "clothes"
    elif elementaryItem["purpose"][i] == "화장하는 경우에 활용":
        elementaryItem["purpose"][i] = "cosmetic"
    else:
        elementaryItem["purpose"][i] = "etc"

elementaryItem.tail()

In [66]:
# 기본항목 전처리 결과 저장
elementaryItem.to_csv("data/elementaryItem.csv")

# 전처리 csv 파일 불러오기
# elementaryItem = pd.read_csv("data/elementaryItem.csv", index_col=0)
# elementaryItem

### 측색 안면부위 설문 결과 전처리

In [96]:
# 측색 안면부위 설문결과 (모집단 분리하지 않은 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
face1 = pd.read_excel("data/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="I, J, K, L")
face1.rename(columns={face1.columns[0]:"region 1",
                          face1.columns[1]:"region 2",
                          face1.columns[2]:"region 3",
                          face1.columns[3]:"region 4"}, inplace=True)

## 조화색 설문 결과 불러오기
face2 = pd.read_excel("data/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="I, J, K, L")
face2.rename(columns={face2.columns[0]:"region 1",
                          face2.columns[1]:"region 2",
                          face2.columns[2]:"region 3",
                          face2.columns[3]:"region 4"}, inplace=True)


## 두 결과 합치기
face = pd.concat([face1, face2], join='outer')


## 인덱스 조정
face.reset_index(inplace=True)
del face['index']

## 중복값 분리 반복문
region1, region2, region3, region4 = [], [], [], []
for i in range(int(face.shape[0])):
    region1.append(face["region 1"][i].split()[0])
    region1.append(face["region 1"][i].split()[1])
    
    region2.append(face["region 2"][i].split()[0])
    region2.append(face["region 2"][i].split()[1])
    
    region3.append(face["region 3"][i].split()[0])
    region3.append(face["region 3"][i].split()[1])
    
    region4.append(face["region 4"][i].split()[0])
    region4.append(face["region 4"][i].split()[1])


## 리스트 값으로 데이터프레임 작성
face_region = pd.DataFrame()
face_region["region 1"] = region1
face_region["region 2"] = region2
face_region["region 3"] = region3
face_region["region 4"] = region4


# 선택항목 별 부위 매칭 (①-cheek, ②-chin, ③-face, ④-eyes, ⑤-forehead)
for i in range(int(face_region.shape[0])):
    ## region1 : 샘플-정우성
    if face_region["region 1"][i] == "①":
        face_region["region 1"][i] = "cheek"
    elif face_region["region 1"][i] == "②":
        face_region["region 1"][i] = "chin"
    elif face_region["region 1"][i] == "③":
        face_region["region 1"][i] = "face"
    elif face_region["region 1"][i] == "④":
        face_region["region 1"][i] = "eyes"
    else:
        face_region["region 1"][i] = "forehead"
    
    ## region2 : 샘플-이효리
    if face_region["region 2"][i] == "①":
        face_region["region 2"][i] = "cheek"
    elif face_region["region 2"][i] == "②":
        face_region["region 2"][i] = "chin"
    elif face_region["region 2"][i] == "③":
        face_region["region 2"][i] = "face"
    elif face_region["region 2"][i] == "④":
        face_region["region 2"][i] = "eyes"
    else:
        face_region["region 2"][i] = "forehead"
    
    ## region3 : 샘플-원빈
    if face_region["region 3"][i] == "①":
        face_region["region 3"][i] = "cheek"
    elif face_region["region 3"][i] == "②":
        face_region["region 3"][i] = "chin"
    elif face_region["region 3"][i] == "③":
        face_region["region 3"][i] = "face"
    elif face_region["region 3"][i] == "④":
        face_region["region 3"][i] = "eyes"
    else:
        face_region["region 3"][i] = "forehead"
    
    ## region1 : 샘플-이영애
    if face_region["region 4"][i] == "①":
        face_region["region 4"][i] = "cheek"
    elif face_region["region 4"][i] == "②":
        face_region["region 4"][i] = "chin"
    elif face_region["region 4"][i] == "③":
        face_region["region 4"][i] = "face"
    elif face_region["region 4"][i] == "④":
        face_region["region 4"][i] = "eyes"
    else:
        face_region["region 4"][i] = "forehead"

face_region.tail()

,region 1,region 2,region 3,region 4
85,face,face,forehead,chin
86,chin,chin,chin,cheek
87,forehead,forehead,eyes,forehead
88,cheek,chin,cheek,cheek
89,chin,forehead,forehead,forehead


In [95]:
# 기본항목 전처리 결과 저장
face_region.to_csv("data/face_region.csv")

# 전처리 csv 파일 불러오기
# face_region = pd.read_csv("data/face_region.csv", index_col=0)
# face_region

### 색고정, 명도/채도변경 팔레트 설문결과 전처리

In [ ]:
# 팔레트 설문결과 (모집단 분리한 항목) 데이터 준비

## 밝은색 설문 결과 불러오기
hueFixed_bright = pd.read_excel("data/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="I, J, K, L")
hueFixed_bright.rename(columns={hueFixed_bright.columns[0]:"region 1",
                          hueFixed_bright.columns[1]:"region 2",
                          hueFixed_bright.columns[2]:"region 3",
                          hueFixed_bright.columns[3]:"region 4"}, inplace=True)

h-f triangle comp 1

## 조화색 설문 결과 불러오기

### 색변경, 명도/채도고정 팔레트 설문결과 전처리

In [ ]:
# 팔레트 설문결과 (모집단 분리한 항목) 데이터 준비

In [ ]:
# personalColor_zzzz= pd.read_excel("data/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (2).xlsx",
#                           sheet_name = 'Sheet1', header=0,na_values = 'NaN',
#                           usecols="C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X")

# pop_seoul.rename(columns={pop_seoul.columns[0]:"구별",
#                           pop_seoul.columns[1]:"인구수",
#                           pop_seoul.columns[2]:"한국인",
#                           pop_seoul.columns[3]:"외국인",
#                           pop_seoul.columns[4]:"고령자"}, inplace=True)

### csv 저장 ###
# 데이터프레임변수명.to_csv("data/저장할이름.csv", encoding="utf-8")
# 뀨 = pd.read_csv("data/파일명.csv", encoding="utf-8", index_col=0)